In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
#import seaborn as sns
import pymongo
from pprint import pprint

In [2]:
client = pymongo.MongoClient()
db = client.metrics

In [3]:
def group_by(df, bycols, agg_map):
    """

    @param df:      DataFrame
    @param bycols:  str or list
                        Column(s) to group by
    @param agg_map: dictionary or list of 2-tuples
                        Mapping from column to aggregate function e.g. [("city", "count"), ("salary", "mean"]
    @return:        DataFrame
                        Flattened dataframe, with multi-level index removed
    """
    grps = []
    if type(bycols) == str:
        bycols = [bycols]

    if type(agg_map) == dict:
        agg_map = agg_map.items()

    for k,v in agg_map:
        grp = df[bycols + [k]].groupby(bycols, ).agg(v)
        grp.reset_index(inplace=True)
        grp["%s(%s)" % (v,k)] = grp[k]
        del grp[k]
        grps.append(grp)

    m = grps[0]
    for grp in grps[1:]:
        m = pd.merge(m, grp, on=bycols, how="inner")
    return m

In [4]:
from bson.son import SON # needed to ensure dictionary is ordered (python default is not)
import hashlib

def hash_feats(fts):
    vals = fts.values
    joined = "|".join(map(lambda s: str(s),vals)).encode('utf-8') 
    return hashlib.sha224(joined).hexdigest()

def get_df_sorted_by_f1score(collection, params=None, filter_cols=True):
    if not params:
        params = []
    if type(params) == str:
        params = params.split(",")
    
    project = {
            "weighted_f1_score":"$WEIGHTED_MEAN_CONCEPT_CODES.f1_score",
            "macro_f1_score":   "$MACRO_F1",
            "micro_f1_score":  "$MICRO_F1.f1_score",
            "micro_recall":    "$MICRO_F1.recall",
            "micro_precision": "$MICRO_F1.precision",
    
    # PARAMETERS            
            "window_size":    "$parameters.window_size",
            "feats":          "$parameters.extractors",
            "count": {        "$size" : "$parameters.extractors" },
            "asof" :          "$asof",
            "_id":1
    }
    
    # No count for HMM
    if "_hmm" in collection.lower():
        del project["count"]
    
    for param in params:
        project[param] = "$parameters." + param

    feats_pipeline = [{
        "$project": project
    },
    {
        "$match":{
            "micro_f1_score": { "$exists" : True }        
        }
    },
    {
        "$sort":{
            "micro_f1_score": -1
        }
    },
    ]
    
    rows = [row for row in db[collection].aggregate(feats_pipeline)]
    df = pd.DataFrame(rows).sort_values("micro_f1_score", ascending=False)
    if params:
        df["hs_params"] = df[params].apply(hash_feats, axis=1)
        
    if filter_cols:
        cols = ["micro_f1_score", "micro_recall" ,"micro_precision", "macro_f1_score" ] + params
        return df[cols]
    return df

In [5]:
def get_window_classifier_results(prefix):
    collections = "WINDOW_CLASSIFIER_BR,WINDOW_CLASSIFIER_LBL_POWERSET_MULTICLASS,WINDOW_CLASSIFIER_MOST_COMMON_TAG_MULTICLASS".split(",")
    dfs = []
    for c in collections:
        col = prefix + c
        print(col)
        df = dict(get_df_sorted_by_f1score(col).iloc[0,:])
        df["Collection_" + prefix[:-1]] = col.replace(prefix,"")
        dfs.append(df)
    return pd.DataFrame(dfs).sort_values("micro_f1_score", ascending=False)

In [7]:
def round_data(df, places=3):
    df_copy = df.copy()
    fmt_str = "{0:." + str(places) + "f}"
    cols = set([v for v in df_copy.columns.values if "micro_" in v])
    for c in cols:
        df_copy[c] = df[c].apply(lambda d: fmt_str.format(d))  
    return df_copy

In [15]:
def compute_macro_metrics(coll):
    for row in db[coll].find({}):
        precision, recall = [],[]
        keys = []
        for k in row.keys():
            if k[0].isdigit():
                keys.append(k)
                code, prec, rec = k, row[k]["precision"], row[k]["recall"]
                precision.append(prec)
                recall.append(rec)
        macro_f1 = row["MACRO_F1"]
        mprec = np.mean(precision)
        mrec =  np.mean(recall)
        est_mf1 = (2 * mprec * mrec) / (mprec + mrec)
        print(coll)
        #print(macro_f1, est_mf1)
        print("macro_f1:{mf1:.4f}\tmprec:{prec:.4f}\tmrec:{rec:.4f}".format(code=code, mf1=macro_f1, prec=mprec, rec=mrec))
        print(",".join(sorted(keys)))
        print("")

In [17]:
for collection in "TEST_CB_TAGGING_VD_WINDOW_CLASSIFIER_MOST_COMMON_TAG_MULTICLASS,TEST_CB_TAGGING_VD_CRF_MOST_COMMON_TAG,TEST_CB_TAGGING_VD_HMM_MOST_COMMON_TAG_MULTICLASS,TEST_CB_TAGGING_VD_AVG_PERCEPTRON_MOST_COMMON_TAG,TEST_CB_TAGGING_VD_RNN_MOST_COMMON_TAG".split(","):
    compute_macro_metrics(collection)

TEST_SC_TAGGING_VD_WINDOW_CLASSIFIER_MOST_COMMON_TAG_MULTICLASS
macro_f1:0.7606	mprec:0.8432	mrec:0.6928
1,11,12,2,3,4,5,50,6

TEST_SC_TAGGING_VD_CRF_MOST_COMMON_TAG
macro_f1:0.7557	mprec:0.8425	mrec:0.6851
1,11,12,2,3,4,5,50,6

TEST_SC_TAGGING_VD_HMM_MOST_COMMON_TAG_MULTICLASS
macro_f1:0.6439	mprec:0.6132	mrec:0.6778
1,11,12,2,3,4,5,50,6

TEST_SC_TAGGING_VD_AVG_PERCEPTRON_MOST_COMMON_TAG
macro_f1:0.7573	mprec:0.8397	mrec:0.6896
1,11,12,2,3,4,5,50,6

TEST_SC_TAGGING_VD_RNN_MOST_COMMON_TAG
macro_f1:0.7793	mprec:0.8619	mrec:0.7111
1,11,12,2,3,4,5,50,6



In [10]:
for collection in "TEST_CB_TAGGING_VD_WINDOW_CLASSIFIER_MOST_COMMON_TAG_MULTICLASS,TEST_CB_TAGGING_VD_CRF_MOST_COMMON_TAG,TEST_CB_TAGGING_VD_HMM_MOST_COMMON_TAG_MULTICLASS,TEST_CB_TAGGING_VD_AVG_PERCEPTRON_MOST_COMMON_TAG,TEST_CB_TAGGING_VD_RNN_MOST_COMMON_TAG".replace("CB","SC").split(","):
    compute_macro_metrics(collection)

TEST_SC_TAGGING_VD_WINDOW_CLASSIFIER_MOST_COMMON_TAG_MULTICLASS
code:2	macro_f1:0.7606	mprec:0.8432	mrec:0.6928

TEST_SC_TAGGING_VD_CRF_MOST_COMMON_TAG
code:2	macro_f1:0.7557	mprec:0.8425	mrec:0.6851

TEST_SC_TAGGING_VD_HMM_MOST_COMMON_TAG_MULTICLASS
code:2	macro_f1:0.6439	mprec:0.6132	mrec:0.6778

TEST_SC_TAGGING_VD_AVG_PERCEPTRON_MOST_COMMON_TAG
code:2	macro_f1:0.7573	mprec:0.8397	mrec:0.6896

TEST_SC_TAGGING_VD_RNN_MOST_COMMON_TAG
code:3	macro_f1:0.7793	mprec:0.8619	mrec:0.7111

